# Chains

# 準備

In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv

# 環境変数の読み込み
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.environ["API_KEY"]

# LCEL：Chainの基本的な使い方

In [2]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

model_name = "gpt-3.5-turbo-instruct"

template = "{subject}を勉強する方法をステップ・バイ・ステップで教えてください"

prompt = PromptTemplate.from_template(template)
model = OpenAI(model=model_name)
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [3]:
chain.invoke({"subject": "英語"})

'\n\nステップ1：目標を設定する\nまずは英語を勉強する目的や目標を設定しましょう。具体的にどの程度のレベルまで英語を話せるようになりたいのか、どのようなスキルを身につけたいのかを明確にすることで、勉強の方針を立てることができます。\n\nステップ2：学習方法を選択する\n英語を勉強する方法は様々あります。自分に合った学習方法を選択することで、より効率的な学習が可能になります。例えば、オンラインの自習プログラムや英会話スクール、教科書や参考書を使用するなど、自分に合った学習方法を'

# Chat Model

In [32]:
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

model_name = "gpt-3.5-turbo-1106"

template = "あなたは{input_language}から{output_language}に翻訳する優秀な翻訳家です。"
human_template = "{text}"

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

model = ChatOpenAI(model=model_name)
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [33]:
chain.invoke({
  "input_language": "英語",
  "output_language": "日本語",
  "text": "I love programming."})

'プログラミングが大好きです。'

# Conversation Chain：会話を続ける方法

In [34]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

model_name = "gpt-3.5-turbo-1106"

model = ChatOpenAI(
    model_name=model_name,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    temperature=1.2,
)

# memoryの使用をLCELで記述する方法が不明なためConversationChainを使う
conversation = ConversationChain(llm=model, memory=ConversationBufferMemory())

In [35]:
while True:
    # ユーザーからの質問を受付
    message = input("メッセージを入力:")

    # 質問が入力されなければ終了
    if message.strip() == "":
        break
    display(f"質問:{message}")

    # 言語モデルに質問
    conversation.invoke(input=message)

print("\n---ご利用ありがとうございました！---")

'質問:こんにちは！'

こんにちは！元気ですか？私の之前のトレーニングから、日本語での会話は得意です。何かお手伝いできることはありますか？

'質問:仙台でおすすめのランチを教えて！'

 仙台でおすすめのランチスポットですね。私が持っているデータによると、仙台市中心部にはたくさんの人気のあるランチレストランがあります。例えば、牛たんや牛ステーキで有名なお店や、海鮮料理を提供する店などがあります。ただし、具体的なおすすめのお店については、私のデータには情報がありません。申し訳ありません。

'質問:ラーメン'

あなたが求めているのはラーメン屋ですね。仙台では有名なラーメン店がたくさんあります。例えば、一風堂や麺場 はなと めんなどがあります。この辺りのお店は地元の人に人気があります。そして、仙台駅周辺にもたくさんのラーメン屋さんがあります。特に味噌ラーメンが有名ですが、塩や醤油ラーメンも美味しいお店がたくさんあります。お好みのラーメンが食べられるお店を見つけることができると思います。
---ご利用ありがとうございました！---


# RAGアーキテクチャ（LCEL)

In [36]:
from langchain.document_loaders.directory import DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# テキストファイルを読込
loader = DirectoryLoader("./data/")
documents = loader.load()

# チャンクに分割
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=512, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Indexの構築
db = Chroma.from_documents(texts, OpenAIEmbeddings())

# 検索（Retriever）の取得
retriever = db.as_retriever()

In [37]:
from langchain_openai import ChatOpenAI

model_name = "gpt-3.5-turbo-1106"

# モデルの作成
model = ChatOpenAI(
    model_name=model_name,
    max_tokens=300,
    temperature=1.2)

In [38]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# プロンプトの作成
prompt = ChatPromptTemplate.from_template("""提供されたコンテキストのみに基づいて次の質問に答えてください:

<コンテキスト>
{context}
</コンテキスト>

Question: {input}""")

# チェーンの作成
chain = (
  {"context": retriever, "input": RunnablePassthrough()}
  | prompt 
  | model 
  | StrOutputParser()
)

In [40]:
# 質問
chain.invoke("キノコストレージについて説明して")

'キノコストレージは、容量の大きなデータストレージ装置を指す架空のIT用語であり、その形状がキノコに似ていることから名付けられました。この装置は非常に小さなサイズでありながら、膨大なデータを保持することができるとされています。また、キノコストレージは高い耐久性を持ち、データの保護と安全性を確保すると説明されます。'

# 対話型RetrievalChain

In [41]:
from langchain.document_loaders import WikipediaLoader
from langchain.document_loaders import PyPDFLoader

# 複数のローダーから読込
loaders = [
    WikipediaLoader(query="SDGs", load_max_docs=3, lang="ja"),
    PyPDFLoader("./data2/20210615_resources_data_guideline_01.pdf")]

documents = []

for loader in loaders:
    documents.extend(loader.load())

In [42]:
documents

[Document(page_content='持続可能な開発目標（じぞくかのうなかいはつもくひょう、英語:  Sustainable Development Goals、略称: SDGs（エスディージーズ））は、2015年9月25日に国連総会で採択された、持続可能な開発のための17の国際目標である。その下に、169の達成基準と232の指標が決められている。\n\nミレニアム開発目標 (MDGs: Millennium Development Goals) が2015年に終了することに伴い、2015年9月25日の国連総会で、『我々の世界を変革する：持続可能な開発のための2030アジェンダ』（Transforming our world: the 2030 Agenda for Sustainable Development、または単に2030 Agendaとも）が採択された。これは、持続可能な開発のために必要不可欠な、向こう15年間の新たな行動計画である。その中で、2030年までに達成するべき持続可能な開発目標 (SDGs) として17の世界的目標と169の達成基準が示された。\n\n\n== 17の目標 ==\n\nSDGsは、以下の17の目標から構成されている。その他、169項目など全文はウィキソースの「持続可能な開発のための2030アジェンダ」を参照。\n\n\n== 進捗状況 ==\n2017年7月 - 国際連合の事務総長であるアントニオ・グテーレスは、SDGsに掲げられている多くの分野の前進が2030年までに達成できるペースをはるかに下回っているとし、前進を加速すべく取り組みを強化する必要があるとする国連報告書を発表した。\n2019年5月 - 国際連合事務総長のアントニオ・グテーレスは、経済社会理事会ECOSOCに提出する「国連持続可能な開発目標SDGs進捗報告書」を公表する。首脳レベルのSDGs進捗状況は4年に1度の発表であるため、2015年に採択されてから初の報告として注目を集める。目標1から17まで各々の課題が発表されたが、全体的に目標達成に至らず、掲げる目標の達成までに課題が山積みしている。\n2020年9月 - 国際連合副事務総長のアミナ・モハメドは、新型コロナウイルス感染症の世界的流行が原因で、貧困撲滅を目指す目標1の取り組みが大幅に減退して

In [43]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# チャンクに分割
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=512, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Indexの構築
db = Chroma.from_documents(texts, OpenAIEmbeddings())

# 検索（Retriever）の取得
retriever = db.as_retriever()

In [62]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

model_name = "gpt-3.5-turbo-1106"

# モデルの作成
model = ChatOpenAI(
    model_name=model_name,
    max_tokens=300,
    temperature=1.2)

# Memoryの作成
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 会話型RetrievalQAチェーンの作成
chain = ConversationalRetrievalChain.from_llm(
    llm=model,
    retriever=retriever,
    max_tokens_limit=4096,
    memory=memory)

In [63]:
# ユーザーからの質問
result = chain.invoke({"question": "SDGsとは何ですか？"})

# 言語モデルからの回答を表示
result["answer"]

c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'SDGsとは、持続可能な開発目標（Sustainable Development Goals）の略称で、国連が2015年に採択した17の目標及び付随する169項目の取り組みから成り立っています。これは2030年までに達成すべき課題を設定し、世界中の国々が貧困削減や教育の促進といった具体的な目標へ取り組むための計画です。'

In [65]:
# ユーザーからの質問
result = chain.invoke({"question": "1つ目の目標について説明してください。出典も示してください"})

# 言語モデルからの回答を表示
result["answer"]

c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDe

'SDGsの1つの目標は、「貧困を終わりにする」という目標です。これは、国際連合によって採択されたものであり、持続可能な開発目標の一環として位置づけられています。この目標は、世界中の貧困層の削減、社会経済格差の解消、そして全体的な豊かさの増進を推進することを目指しています。\n\n（出典）2015年のエコノミスト\n\nエコノミストの解説によると、他の16の目標はすべてSDG1「貧困を終わらせる」の達成を条件としている可能性があり、これは非常に短い目標リストのトップにあるべきものであったとしています。'

In [66]:
# ユーザーからの質問
result = chain.invoke({"question": "オープンデータの定義とは？"})

# 言語モデルからの回答を表示
result["answer"]

c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDe

'オープンデータとは、政府や行政機関が保有するデータを誰もが自由に利用・再利用できるように公開することです。政策の基盤となったデータを含むすべての府省庁のデータを原則として公開し、利用者のニーズを反映しやすい形で公開されることも含まれています。'

In [67]:
# 保存されているメッセージを取得
buffer = memory.load_memory_variables({})
print(buffer)

{'chat_history': [HumanMessage(content='SDGsとは何ですか？'), AIMessage(content='SDGsとは、持続可能な開発目標（Sustainable Development Goals）の略称で、国連が2015年に採択した17の目標及び付随する169項目の取り組みから成り立っています。これは2030年までに達成すべき課題を設定し、世界中の国々が貧困削減や教育の促進といった具体的な目標へ取り組むための計画です。'), HumanMessage(content='1つ目の目標について説明してください。出典も示してください'), AIMessage(content='申し訳ございませんが、そのような情報についてはお手伝いできません。'), HumanMessage(content='1つ目の目標について説明してください。出典も示してください'), AIMessage(content='SDGsの1つの目標は、「貧困を終わりにする」という目標です。これは、国際連合によって採択されたものであり、持続可能な開発目標の一環として位置づけられています。この目標は、世界中の貧困層の削減、社会経済格差の解消、そして全体的な豊かさの増進を推進することを目指しています。\n\n（出典）2015年のエコノミスト\n\nエコノミストの解説によると、他の16の目標はすべてSDG1「貧困を終わらせる」の達成を条件としている可能性があり、これは非常に短い目標リストのトップにあるべきものであったとしています。'), HumanMessage(content='オープンデータの定義とは？'), AIMessage(content='オープンデータとは、政府や行政機関が保有するデータを誰もが自由に利用・再利用できるように公開することです。政策の基盤となったデータを含むすべての府省庁のデータを原則として公開し、利用者のニーズを反映しやすい形で公開されることも含まれています。')]}


# Summarize Chain：文章の要約

In [68]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

# PDFの読込
loader = PyPDFLoader("./data2/20210615_resources_data_guideline_01.pdf")
texts = loader.load()

# チャンクに分割
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=512, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [69]:
from langchain_openai import ChatOpenAI

model_name = "gpt-3.5-turbo-1106"

# モデルの作成
model = ChatOpenAI(
    model_name=model_name,
    max_tokens=300,
    temperature=1.2)

In [74]:
from langchain.chains.summarize import load_summarize_chain

summarize_chain = load_summarize_chain(llm=model, chain_type="stuff", verbose=False)
result = summarize_chain.invoke(texts)

c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [76]:
result["output_text"]

'The Sustainable Development Goals (SDGs) were established by the United Nations to address sustainable development through 17 international goals, with 169 targets and 232 indicators. They were adopted in 2015 as part of the 2030 Agenda for Sustainable Development, following the conclusion of the Millennium Development Goals. However, progress towards achieving the SDGs has been slower than anticipated, as indicated by reports from UN Secretary-General Antonio Guterres. In Japan, the media and small businesses have actively participated in SDG-related initiatives, such as promoting solar energy and insect consumption. However, there has also been criticism and pushback against some SDG initiatives, such as incorporating insects into school lunches due to food sovereignty concerns. The adoption and integration of SDGs has also been a priority for the Japanese government, but costly implementations and negative impacts related to SDGs are causing "SDGs fatigue" to proliferate throughout

In [77]:
summarize_chain = load_summarize_chain(llm=model, chain_type="map_reduce", verbose=False)
result = summarize_chain.invoke(texts)

c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDe

In [78]:
result["output_text"]

'The text discusses the establishment of open data guidelines in Japan, focusing on promoting the use of government data through an open data catalog site. It outlines the process of releasing data, considerations for limited release, and requirements for updating and maintaining data. The document also provides guidelines for creating and publishing data in accessible formats. The government is also taking steps to involve public and private sectors in discussing and utilizing open data, and encouraging local governments and businesses to make their data readily available. It emphasizes the importance of promoting open data in various fields and making research data funded by public funds public.'

In [79]:
from langchain import PromptTemplate

prompt = PromptTemplate(
  input_variables=["output_text"], 
  template="{output_text}\n上記の文章を日本語訳してください")

chain = summarize_chain | prompt | model | StrOutputParser()
result = chain.invoke({"input_documents": texts})
print(result)

c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDe

持続可能な開発目標（SDGs）は、2015年に国連で採択された17の国際目標であり、持続可能な開発を促進するためのものです。日本では、ビジネスマネジメントや政府政策にSDGsを取り込むことに積極的な取り組みがありますが、対立する達成基準や「SDGs疲れ」といった課題が観察されています。政府は各省庁が保有するデータのリストを作成し、データ共有のための標準フォーマットやルールの使用を奨めることを目指しています。また、公的資金によって資金援助される研究データへの公衆へのアクセスの重要性を強調しています。国際および国内の取り組み全体で、社会、経済、環境の課題を


# Sequential Chain

### SimpleSequentialChain

In [81]:
from langchain import PromptTemplate, LLMChain
from langchain.chains import LLMCheckerChain, SimpleSequentialChain

llm = OpenAI(temperature=0.7)

# 1つ目のChain
template = "{subject}に関する問題を作成してください。答えは作成しないでください"
llm_chain = LLMChain(
    llm=model,
    prompt=PromptTemplate.from_template(template),
)

# 2つ目のChain
check_chain = LLMCheckerChain.from_llm(
    llm=llm)

# Chain
overall_chain = SimpleSequentialChain(
    chains=[llm_chain, check_chain],
    verbose=True)

In [82]:
# Simple Sequential Chainの実行
overall_chain.invoke("動物")



> Entering new SimpleSequentialChain chain...


c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


1. 哺乳類と魚類の違いは何ですか？
2. 動物の群れの中での役割は何ですか？
3. 動物の保護のために何ができるでしょうか？
4. 哺乳類の中で一番速い動物は何ですか？
5. 動物の寿命は何によって決まりますか？


> Entering new SequentialChain chain...


c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangCh


> Finished chain.

1. 哺乳類と魚類の違いは、哺乳類は胎生であるのに対し、魚類は卵生であることが最も大きな違いです。また、哺乳類は体温調節ができるため、寒冷な環境でも生きることができますが、魚類は体温調節ができないため、温暖な環境を好む傾向があります。
2. 動物の群れの中での役割は、その動物の種類によって異なります。例えば、ライオンの群れではメスが狩りを担当し、オスが群れを守る役割を担います。
3. 動物の保護のためにできることは

> Finished chain.


{'input': '動物',
 'output': '\n1. 哺乳類と魚類の違いは、哺乳類は胎生であるのに対し、魚類は卵生であることが最も大きな違いです。また、哺乳類は体温調節ができるため、寒冷な環境でも生きることができますが、魚類は体温調節ができないため、温暖な環境を好む傾向があります。\n2. 動物の群れの中での役割は、その動物の種類によって異なります。例えば、ライオンの群れではメスが狩りを担当し、オスが群れを守る役割を担います。\n3. 動物の保護のためにできることは'}

### SequentialChain

In [83]:
from langchain import PromptTemplate

# 1つ目のChain
template1 = "{theme}に関する{target}向けのブログ記事のタイトルを考えてください。"
llm_chain1 = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(template1),
    output_key="title"
)

# 2つ目のChain
template2 = "ブログ記事「{title}」の見出しを考えてください。"
llm_chain2 = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(template2),
    output_key="headline"
)

# Chain
from langchain.chains import SequentialChain
overall_chain = SequentialChain(
    chains=[llm_chain1, llm_chain2],
    input_variables=["theme", "target"],
    output_variables=["title", "headline"],
    verbose=True)

In [84]:
# Chainの実行
overall_chain({
    "theme":"観光",
    "target": "日本の富裕層"})

c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


c:\Sample2\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



> Finished chain.


{'theme': '観光',
 'target': '日本の富裕層',
 'title': '\n\n1. "日本の隠れた絶景を巡る贅沢な旅"\n2. "高級ホテルで贅沢に過ごす東京観光ガイド"\n3. "日本の伝統文化を体験する贅沢な旅"\n4. "日本の美食を堪能する豪華グルメツアー"\n5. "伝統と現代が融合する京都の魅力に迫る"\n6. "日本の秘境を訪ねるヘリコプターツアー"\n7. "最高級の温泉宿で贅沢な癒しの旅"\n8. "文化的な都市・奈良を贅沢に満喫する方法"\n9. "日本の美しい自然を楽しむラグジュアリーアウトドア旅"\n10. "',
 'headline': '\n\n1. "日本の隠れた極上の絶景を巡る贅沢な旅"\n2. "東京の高級ホテルで贅沢に過ごす至福の観光ガイド"\n3. "伝統文化を贅沢に体験する日本の旅"\n4. "日本の極上グルメを堪能する豪華なグルメツアー"\n5. "古都・京都の魅力を贅沢に堪能する旅"\n6. "日本の秘境を贅沢に訪ねるヘリコプターツアー"\n7. "極上の温泉宿で贅沢な癒しの旅を満喫する"\n8. "奈良の文化を贅沢に体験する旅の方法"\n9. "日本の美しい自然をラグジュ'}